# ES(1+1) with 1/5 Rule

Name: Batyr Kenzheakhmetov

ID: 000801267

Honor Code: "I have neither given nor received unauthorized aid in completing this work, nor have I presented someone else's work as my own."

In [ ]:
import kagglehub as kaggle
import pandas as pd
import numpy as np
import os

path = kaggle.dataset_download("cherngs/heart-disease-cleveland-uci")
print("Path to dataset files: ", path)

path = "/root/.cache/kagglehub/datasets/cherngs/heart-disease-cleveland-uci/versions/1"
print("Files in the directory:", os.listdir(path))

diabetes_data = pd.read_csv(os.path.join(path, 'heart_cleveland_upload.csv'))
print(diabetes_data.shape)
diabetes_data.head()

Path to dataset files:  /root/.cache/kagglehub/datasets/cherngs/heart-disease-cleveland-uci/versions/1
Files in the directory: ['heart_cleveland_upload.csv']
(297, 14)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,condition
0,69,1,0,160,234,1,2,131,0,0.1,1,1,0,0
1,69,0,0,140,239,0,0,151,0,1.8,0,2,0,0
2,66,0,0,150,226,0,0,114,0,2.6,2,0,0,0
3,65,1,0,138,282,1,2,174,0,1.4,1,1,0,1
4,64,1,0,110,211,0,2,144,1,1.8,1,0,0,0


In [ ]:
diabetes_data.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,condition
count,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000
mean,54.542088,0.676768,2.158249,131.693603,247.350168,0.144781,0.996633,149.599327,0.326599,1.055556,0.602694,0.676768,0.835017,0.461279
std,9.049736,0.468500,0.964859,17.762806,51.997583,0.352474,0.994914,22.941562,0.469761,1.166123,0.618187,0.938965,0.956690,0.499340
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,48.000000,0.000000,2.000000,120.000000,211.000000,0.000000,0.000000,133.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,56.000000,1.000000,2.000000,130.000000,243.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,0.000000,0.000000
75%,61.000000,1.000000,3.000000,140.000000,276.000000,0.000000,2.000000,166.000000,1.000000,1.600000,1.000000,1.000000,2.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,3.000000,2.000000,1.000000


In [ ]:
from sklearn.model_selection import train_test_split

#X = diabetes_data.iloc[:,0:13] #0-13 columns are dependent variables
X = diabetes_data.to_numpy()[:,0:13]
#y = diabetes_data.iloc[:,13]   #13 column is independent variable (label)
y = diabetes_data.to_numpy()[:,13]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=None)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(y_test)

(222, 13)
(222,)
(75, 13)
(75,)
[0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0.
 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1.
 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0.
 1. 0. 1.]


In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Input
from keras import optimizers
from keras import callbacks

act_func = ('relu', 'elu', 'sigmoid', 'tanh')
optimz = ('SGD', 'RMSprop', 'Adam')


def RecFunc(param):
  model = Sequential()
  model.add(Input(shape=(13,)))
  for _ in range(2):
    model.add(Dense(int(param[0]), activation=act_func[round(param[3])]))
  model.add(Dense(1, activation='sigmoid'))

  if ( param[4] == 0 ):
    optmzr = optimizers.SGD(learning_rate=param[1]) # stochastic gradient decent
  elif ( param[4] == 1 ):
    optmzr = optimizers.RMSprop(learning_rate=param[1])
  elif ( param[4] == 2 ):
    optmzr = optimizers.Adam(learning_rate=param[1])

  model.compile(optimizer=optmzr, loss='binary_crossentropy', metrics=['accuracy'])
  cb_list = [
      callbacks.EarlyStopping(monitor='val_loss', patience=4,
                              restore_best_weights=True),
    ]

  model.fit(x_train, y_train, epochs=250, batch_size=int(param[2]), verbose=0,
            validation_data=(x_test, y_test),
            callbacks=cb_list)

  (loss, acc) = model.evaluate(x_test, y_test, verbose=0)

  return loss, acc

In [ ]:
# to test a set of parameters
pr = np.empty(5)
pr[0] = 8   # num hidden neuron
pr[1] = 0.01 # lr
pr[2] = 4   # batch size
pr[3] = 0   # act_func
pr[4] = 0   # optimizer
print(RecFunc(pr))

(0.6862969994544983, 0.5866666436195374)


In [ ]:
# Constants
MaxGen = 1000
MaxTrial = 5
numVar = 5

# acceptable range for each hyper parameter
low0 = 8  # number of hidden neurons
high0 = 512
low1  = 0.01 # smallest learning rate
high1 = 1.5
low2  = 1 # min batch size
high2 = 16

totgen = 0
foundCnt = 0  # acceptable solution found
objfunc = RecFunc

# for 1/5 rule
stepSize_i = 0.82 # initial stepsize
stepSize_r = 0.82 # stepsize change ratio.
                  # 0.82 was used previously by Hans-Paul Schwefel, a PhD student of Rechenberg


print(f"[#neurons, lr, bsize, #epochs, actfun, lossfun, optmzr]")

def print_HP_found(x, eval, acc):
  print(f"#neurons={int(x[0])}, lr={x[1]:.3f}, bsize={int(x[2])}, actF={act_func[int(x[3])]}, optim={optimz[int(x[4])]}")
  print(f"Accuracy={acc:.5f}")
  print(f"Loss={eval:.5f}")

for trial in range(0, MaxTrial):
    xp = np.empty(numVar) # parent
    p_val = 0
    xo = np.empty(numVar) # offspring
    o_val = 0

    successCnt = 0;
    WindowSize = 20


    print(f"************************** Trial # = {trial+1}")

    # initialize hyper parameters
    xp[0] = round(np.random.randint(low0, high0))
    xp[1] = round(np.random.uniform(low1, high1), 2) # init learning rate
    xp[2] = 1 # batch size
    xp[3] = 0 # sigmoid
    xp[4] = 1 # optimizer

    p_val, p_acc = objfunc(xp)     # evaluate the parent

    stepSize = stepSize_i

    for g in range(1, MaxGen+1):
        if (g % WindowSize) == 0: # update stepsize
            if successCnt > (WindowSize * 0.2):
                stepSize = stepSize / stepSize_r #increase
            elif successCnt < (WindowSize * 0.2):
                stepSize = stepSize * stepSize_r #decrease
            #else do not change stepSize
            #print(successCnt, stepSize)
            successCnt = 0

        def mutate_activation_function(xp, stepSize):
            # 30% chance to mutate the activation function
            if np.random.random() < 0.3:
                # Randomly select a new activation function (excluding the current one)
                current_act = int(xp[3])
                new_act = np.random.choice([i for i in range(len(act_func)) if i != current_act])
                xp[3] = new_act
            return xp

        def mutate_optimizer(xp, stepSize):
            # 30% chance to mutate the optimizer
            if np.random.random() < 0.3:
                # Randomly select a new optimizer (excluding the current one)
                current_opt = int(xp[4])
                new_opt = np.random.choice([i for i in range(len(optimz)) if i != current_opt])
                xp[4] = new_opt
            return xp

        # Existing mutation for continuous parameters
        def mutate_continuous(x, index, low, high, stepSize):
            x[index] = x[index] + round(np.random.normal(0.0, stepSize))
            if x[index] < low: x[index] = low
            elif x[index] > high: x[index] = high
            return x

        def mutate_neurons(x, index, low, high, step_size):
            # Mutate with a larger step size for neurons
            x[index] = int(x[index] + int(np.round(np.random.normal(0.0, stepSize))))
            if x[index] < low:
                x[index] = low
            elif x[index] > high:
                x[index] = high
            return x

        # Example of integrating this into the main loop where we mutate each hyperparameter
        def mutate_hyperparameters(xp, stepSize):
            # Mutate number of hidden neurons
            xp = mutate_neurons(xp, 0, low0, high0, stepSize)

            # Mutate learning rate
            xp = mutate_continuous(xp, 1, low1, high1, stepSize)

            # Mutate batch size
            xp = mutate_continuous(xp, 2, low2, high2, stepSize)

            # Mutate activation function using the custom operator
            xp = mutate_activation_function(xp, stepSize)

            # Mutate optimizer using the custom operator
            xp = mutate_optimizer(xp, stepSize)

            return xp

        # Mutate the hyperparameters using the new function
        xo = xp.copy()  # create offspring by copying the parent
        xo = mutate_hyperparameters(xo, stepSize)  # mutate the offspring

        o_val, o_acc = objfunc(xo)  # evaluate offspring

        # select
        if o_val < p_val:
            #xp = xo # No! must maintain two separate spaces
            xp = xo.copy()
            p_val = o_val
            p_acc = o_acc
            successCnt += 1;


        if p_val < 0.7: # we assume minima is zero.
            print(f"Acceptable solution found after {g} iterations:")
            #print(f"{xp}, {p_val:.5f}")
            print_HP_found(xp, p_val, p_acc)
            totgen += g
            foundCnt += 1
            break # cannot use while since it may not find acceptable minia

print(f"System Success = {foundCnt/MaxTrial*100}%")
totgen += (MaxTrial-foundCnt)*MaxGen
print(f"Avgerage # of generations used = {(totgen/MaxTrial):,.0f}")

[#neurons, lr, bsize, #epochs, actfun, lossfun, optmzr]
************************** Trial # = 1
Acceptable solution found after 1 iterations:
#neurons=42, lr=0.720, bsize=1, actF=relu, optim=RMSprop
Accuracy=0.58667
Loss=0.67805
************************** Trial # = 2
Acceptable solution found after 1 iterations:
#neurons=424, lr=1.040, bsize=1, actF=relu, optim=RMSprop
Accuracy=0.58667
Loss=0.67810
************************** Trial # = 3
Acceptable solution found after 1 iterations:
#neurons=153, lr=1.410, bsize=1, actF=relu, optim=RMSprop
Accuracy=0.58667
Loss=0.67807
************************** Trial # = 4
Acceptable solution found after 1 iterations:
#neurons=296, lr=0.010, bsize=2, actF=relu, optim=RMSprop
Accuracy=0.86667
Loss=0.35427
************************** Trial # = 5
Acceptable solution found after 1 iterations:
#neurons=338, lr=0.710, bsize=1, actF=relu, optim=RMSprop
Accuracy=0.58667
Loss=0.67806
System Success = 100.0%
Avgerage # of generations used = 1


This model has the same number of neurons in two hidden layers. I also set the acceptable loss value to < 0.7. As a result, during the fourth attempt, I got an excellent result, but most likely this is a coincidence. During the creation of the second model, I added a new separate parameter to determine the number of neurons in the second hidden layer so that the values ​​​​are different and changed the acceptable loss value to < 0.4. As a result, all attempts can be considered successful!

In [ ]:
def RecFunc(param):
  model = Sequential()
  model.add(Input(shape=(13,)))
  model.add(Dense(int(param[0]), activation=act_func[round(param[3])]))
  model.add(Dense(int(param[5]), activation=act_func[round(param[3])]))
  model.add(Dense(1, activation='sigmoid'))

  if ( param[4] == 0 ):
    optmzr = optimizers.SGD(learning_rate=param[1]) # stochastic gradient decent
  elif ( param[4] == 1 ):
    optmzr = optimizers.RMSprop(learning_rate=param[1])
  elif ( param[4] == 2 ):
    optmzr = optimizers.Adam(learning_rate=param[1])

  model.compile(optimizer=optmzr, loss='binary_crossentropy', metrics=['accuracy'])
  cb_list = [
      callbacks.EarlyStopping(monitor='val_loss', patience=4,
                              restore_best_weights=True),
    ]

  model.fit(x_train, y_train, epochs=250, batch_size=int(param[2]), verbose=0,
            validation_data=(x_test, y_test),
            callbacks=cb_list)

  (loss, acc) = model.evaluate(x_test, y_test, verbose=0)

  return loss, acc


# Constants
MaxGen = 1000
MaxTrial = 5
numVar = 6

# acceptable range for each hyper parameter
low0 = 8  # number of hidden neurons
high0 = 512
low1  = 0.01 # smallest learning rate
high1 = 1.5
low2  = 1 # min batch size
high2 = 16

totgen = 0
foundCnt = 0  # acceptable solution found
objfunc = RecFunc

# for 1/5 rule
stepSize_i = 0.82 # initial stepsize
stepSize_r = 0.82 # stepsize change ratio.
                  # 0.82 was used previously by Hans-Paul Schwefel, a PhD student of Rechenberg


print(f"[#neurons, lr, bsize, #epochs, actfun, lossfun, optmzr]")

def print_HP_found(x, eval, acc):
  print(f"#neurons in 1 layer={int(x[0])}, #neurons in 2 layer={int(x[5])}, lr={x[1]:.3f}, bsize={int(x[2])}, actF={act_func[int(x[3])]}, optim={optimz[int(x[4])]}")
  print(f"Accuracy={acc:.5f}")
  print(f"Loss={eval:.5f}")

for trial in range(0, MaxTrial):
    xp = np.empty(numVar) # parent
    p_val = 0
    xo = np.empty(numVar) # offspring
    o_val = 0

    successCnt = 0;
    WindowSize = 20


    print(f"************************** Trial # = {trial+1}")

    # initialize hyper parameters
    xp[0] = np.random.randint(low0, high0)
    xp[1] = round(np.random.uniform(low1, high1), 2) # init learning rate
    xp[2] = 1 # batch size
    xp[3] = 0 # sigmoid
    xp[4] = 1 # optimizer
    xp[5] = np.random.randint(low0, high0)

    p_val, p_acc = objfunc(xp)     # evaluate the parent

    stepSize = stepSize_i

    for g in range(1, MaxGen+1):
        if (g % WindowSize) == 0: # update stepsize
            if successCnt > (WindowSize * 0.2):
                stepSize = stepSize / stepSize_r #increase
            elif successCnt < (WindowSize * 0.2):
                stepSize = stepSize * stepSize_r #decrease
            #else do not change stepSize
            #print(successCnt, stepSize)
            successCnt = 0

        def mutate_activation_function(xp, stepSize):
            # 30% chance to mutate the activation function
            if np.random.random() < 0.3:
                # Randomly select a new activation function (excluding the current one)
                current_act = int(xp[3])
                new_act = np.random.choice([i for i in range(len(act_func)) if i != current_act])
                xp[3] = new_act
            return xp

        def mutate_optimizer(xp, stepSize):
            # 30% chance to mutate the optimizer
            if np.random.random() < 0.3:
                # Randomly select a new optimizer (excluding the current one)
                current_opt = int(xp[4])
                new_opt = np.random.choice([i for i in range(len(optimz)) if i != current_opt])
                xp[4] = new_opt
            return xp

        # Existing mutation for continuous parameters
        def mutate_continuous(x, index, low, high, stepSize):
            x[index] = x[index] + round(np.random.normal(0.0, stepSize))
            if x[index] < low: x[index] = low
            elif x[index] > high: x[index] = high
            return x

        def mutate_neurons(x, index, low, high, step_size):
            # Mutate with a larger step size for neurons
            x[index] = int(x[index] + int(np.round(np.random.normal(0.0, stepSize))))
            if x[index] < low:
                x[index] = low
            elif x[index] > high:
                x[index] = high
            return x

        # Example of integrating this into the main loop where we mutate each hyperparameter
        def mutate_hyperparameters(xp, stepSize):
            # Mutate number of hidden neurons
            xp = mutate_neurons(xp, 0, low0, high0, stepSize)
            xp = mutate_neurons(xp, 5, low0, high0, stepSize)

            # Mutate learning rate
            xp = mutate_continuous(xp, 1, low1, high1, stepSize)

            # Mutate batch size
            xp = mutate_continuous(xp, 2, low2, high2, stepSize)

            # Mutate activation function using the custom operator
            xp = mutate_activation_function(xp, stepSize)

            # Mutate optimizer using the custom operator
            xp = mutate_optimizer(xp, stepSize)

            return xp

        # Mutate the hyperparameters using the new function
        xo = xp.copy()  # create offspring by copying the parent
        xo = mutate_hyperparameters(xo, stepSize)  # mutate the offspring

        o_val, o_acc = objfunc(xo)  # evaluate offspring

        # select
        if o_val < p_val:
            #xp = xo # No! must maintain two separate spaces
            xp = xo.copy()
            p_val = o_val
            p_acc = o_acc
            successCnt += 1;


        if p_val < 0.4: # we assume minima is zero.
            print(f"Acceptable solution found after {g} iterations:")
            #print(f"{xp}, {p_val:.5f}")
            print_HP_found(xp, p_val, p_acc)
            totgen += g
            foundCnt += 1
            break # cannot use while since it may not find acceptable minia

print(f"System Success = {foundCnt/MaxTrial*100}%")
totgen += (MaxTrial-foundCnt)*MaxGen
print(f"Avgerage # of generations used = {(totgen/MaxTrial):,.0f}")

[#neurons, lr, bsize, #epochs, actfun, lossfun, optmzr]
************************** Trial # = 1
Acceptable solution found after 2 iterations:
#neurons in 1 layer=489, #neurons in 2 layer=263, lr=0.010, bsize=1, actF=relu, optim=RMSprop
Accuracy=0.86667
Loss=0.23965
************************** Trial # = 2
Acceptable solution found after 18 iterations:
#neurons in 1 layer=72, #neurons in 2 layer=21, lr=0.010, bsize=1, actF=relu, optim=RMSprop
Accuracy=0.89333
Loss=0.26941
************************** Trial # = 3
Acceptable solution found after 23 iterations:
#neurons in 1 layer=423, #neurons in 2 layer=195, lr=0.010, bsize=2, actF=relu, optim=RMSprop
Accuracy=0.80000
Loss=0.39779
************************** Trial # = 4
Acceptable solution found after 7 iterations:
#neurons in 1 layer=480, #neurons in 2 layer=12, lr=0.010, bsize=3, actF=elu, optim=Adam
Accuracy=0.88000
Loss=0.25298
************************** Trial # = 5
Acceptable solution found after 170 iterations:
#neurons in 1 layer=238, #

In [ ]:
import pandas as pd

# Data based on your provided results
data = {
    "Run #": [1, 2, 3, 4, 5],
    "# ns 1st h-layer": [489, 72, 423, 480, 238],
    "# ns 2nd h-layer": [263, 21, 195, 12, 9],
    "# ns 3rd h-layer": ["N/A", "N/A", "N/A", "N/A", "N/A"],  # Add 'N/A' since only two layers were used
    "Learn. rate": [0.010, 0.010, 0.010, 0.010, 0.010],
    "Batch size": [1, 1, 2, 3, 3],
    "Act. func": ["relu", "relu", "relu", "elu", "elu"],
    "Optmzr": ["RMSprop", "RMSprop", "RMSprop", "Adam", "Adam"],
    "Loss": [0.23965, 0.26941, 0.39779, 0.25298, 0.31789],
    "Accrcy": [0.86667, 0.89333, 0.80000, 0.88000, 0.89333]
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Display the table
df.style.set_properties(**{'background-color': '#b3d9ff', 'color': 'black', 'border-color': 'black'})


,Run #,# ns 1st h-layer,# ns 2nd h-layer,# ns 3rd h-layer,Learn. rate,Batch size,Act. func,Optmzr,Loss,Accrcy
0,1,489,263,N/A,0.010000,1,relu,RMSprop,0.239650,0.866670
1,2,72,21,N/A,0.010000,1,relu,RMSprop,0.269410,0.893330
2,3,423,195,N/A,0.010000,2,relu,RMSprop,0.397790,0.800000
3,4,480,12,N/A,0.010000,3,elu,Adam,0.252980,0.880000
4,5,238,9,N/A,0.010000,3,elu,Adam,0.317890,0.893330
